## PRE-REQUISITES

Installing datasets, connecting to Google Drive (where the datasets have been stored)

In [1]:
! pip install datasets transformers[sentencepiece] sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.6 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/
%cd gcm1job

/content/drive/MyDrive
/content/drive/MyDrive/gcm1job


## Processing and filtering out records

In [5]:
def process_record(df):
    # Applying conditional checks

    if((df["unicode_mix_latin_devnagri"] == True) and (df["CM_not_same_as_l1_or_l2"] == True)):
      if(df["CMI_unicode_based_LID"] > 30):
        if(df["alignment_value"] == 0):
          df["length"] = len(df["CM_candidates"].split())

          if(df["length"] > 7):
            # calcing length of sentence L1 and L2
            df["L1_length"] = len(df["L1"].split())
            df["L2_length"] = len(df["L2"].split())

            # calcing differences in sentence length wrt each other
            df["L1_CM_length_diff"] = abs(df["L1_length"] - df["length"])
            df["L2_CM_length_diff"] = abs(df["L2_length"] - df["length"])
            df["L1_L2_length_diff"] = abs(df["L1_length"] - df["L2_length"])

            if((df["L1_CM_length_diff"] < 4) and (df["L2_CM_length_diff"] < 4)):
              if(df["L1_L2_length_diff"] < 4):
                return True
    return False


## Writing filtered records to json file

In [ ]:
import os, sys, re
from collections import Counter
import pandas as pd
import json
from tqdm import tqdm
tqdm.pandas()

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
pd.set_option('display.float_format', '{:20,.2f}'.format)
pd.set_option('display.max_colwidth', None)

records = []
total_lines = 0
processed_lines = 1
fw_en = open("en_enhidata.json", 'a', encoding='utf8')
fw_hi = open("hi_enhidata.json", 'a', encoding='utf8')
cwd = os.getcwd()
for ind,file in enumerate(os.listdir(cwd)):
    if ".json" in file and file[:2] == "ec":
        print(file)
        num_lines = sum(1 for line in open(file,'r'))
        total_lines += num_lines
        f = open(file, 'r')
        for line in tqdm(f, total=num_lines):
          jsline = json.loads(line)
          if(process_record(jsline)):
            en_enhi = {"id": str(processed_lines), "translation" : {"en":jsline["L2"], "en-hi":jsline["CM_candidates"]}}
            hi_enhi = {"id": str(processed_lines), "translation" : {"hi":jsline["L1"], "en-hi":jsline["CM_candidates"]}}

            fw_en.write(json.dumps(en_enhi))
            fw_hi.write(json.dumps(hi_enhi))

            fw_en.write("\n")
            fw_hi.write("\n")
            processed_lines+=1

        print(processed_lines)

print(processed_lines)

## Converting to huggingface dataset and loading to hub

In [6]:
from datasets import Dataset, load_dataset
dataset_en_enhi = load_dataset('json', data_files={'en_enhidata.json'})
print(f"Train dataset size: {len(dataset_en_enhi['train'])}")

dataset_hi_enhi = load_dataset('json', data_files={'hi_enhidata.json'})
print(f"Train dataset size: {len(dataset_hi_enhi['train'])}")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-033d26ec76a2246d/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Train dataset size: 1253785


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-5a8e5e4d751991b9/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Train dataset size: 1254032


In [7]:
dataset_hi_enhi["train"]

Dataset({
    features: ['id', 'translation'],
    num_rows: 1254032
})

In [ ]:
from datasets import DatasetDict

dataset_en_enhi = DatasetDict({
    'train': dataset_en_enhi['train']})

dataset_hi_enhi = DatasetDict({
    'train': dataset_hi_enhi['train']})

'''
If you want to make dictionary with train, test and validation: use the below script:

dataset_en_enhi_train_testvalid = dataset_en_enhi['train'].train_test_split(test_size=0.1)
dataset_en_enhi_test_valid = dataset_en_enhi_train_testvalid['test'].train_test_split(test_size=0.5)

dataset_hi_enhi_train_testvalid = dataset_hi_enhi['train'].train_test_split(test_size=0.1)
dataset_hi_enhi_test_valid = dataset_hi_enhi_train_testvalid['test'].train_test_split(test_size=0.5)

dataset_en_enhi = DatasetDict({
    'train': dataset_en_enhi_train_testvalid['train'],
    'test': dataset_en_enhi_test_valid['test'],
    'validation': dataset_en_enhi_test_valid['train']})

dataset_hi_enhi = DatasetDict({
    'train': dataset_hi_enhi_train_testvalid['train'],
    'test': dataset_hi_enhi_test_valid['test'],
    'validation': dataset_hi_enhi_test_valid['train']})

'''

In [9]:
dataset_en_enhi

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 1253785
    })
})

In [10]:
dataset_hi_enhi

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 1254032
    })
})

In [13]:
dataset_en_enhi.push_to_hub("kapilrk04/codemix-en_enhi", private=True)
dataset_hi_enhi.push_to_hub("kapilrk04/codemix-hi_enhi", private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1255 [00:00<?, ?ba/s]